<a href="https://colab.research.google.com/github/MennoLPomp/MennoLPomp/blob/main/TTCC_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from google.colab import files
from scipy.io import loadmat
import matplotlib.pyplot as plt

import networkx as nx

pd.set_option('display.max_rows', None)

In [ ]:
files.upload(); # upload K9.mat here

Saving K9.mat to K9.mat


In [ ]:
A = loadmat('K9.mat')['K7'].T - 1 # start patient count from 0

# Exercise (a)

### Auxiliary functions

In [ ]:
def reconstruct_G_fully(G, matching, passive_matching, unassigned, players):
        # reconstruct graph
        remaining_players = players.difference(matching.keys())
        G = nx.create_empty_copy(G)
        # add passive player's assignments
        G.add_edges_from(passive_matching.items())
        for player in remaining_players:
            for preference in A[:, player]:
                if preference in unassigned:
                    G.add_edge(player, preference)
                    break
        return G
                    
def resolve_cycle(G, matching, unassigned, cycle):
    M = len(cycle)
    for j in range(M):
        matching[cycle[j]] = cycle[(j+1) % M]
        if cycle[j] in unassigned:
            unassigned.remove(cycle[j])
            
    return matching, unassigned
    
def find_w_chains(G, w, matching):
    '''Finds all w-chains that contain new matchings'''
    
    # add virtual edges from w to all players such that w-chains become cycle
    G.add_edges_from([(w, x) for x in G.nodes])

    chains = []
    for chain in nx.simple_cycles(G):
        # check if chain is novel, i.e. we aren't stuck in an infinite loop
        if len(set(chain).difference(matching.keys()).difference({w})) > 0:
            chains.append(chain)
    
    # remove virtual edges
    G.remove_edges_from([(w, x) for x in G.nodes])
    return chains

def rule_e(matching, players, chains):
    '''See slides lecture 2'''
    max_chains_len = len(max(chains, key=len))
    # only select chains of max length
    max_chains = np.array([c for c in chains if len(c) == max_chains_len])
    # priority over remaining players
    remaining_players = players.difference(matching.keys())
    priority = sorted(list(remaining_players.intersection(set(max_chains.flatten()))))
    i = 0                                          
    while len(max_chains) > 1:
        candinates = np.apply_along_axis(lambda x: priority[i] in x, 1, max_chains)
        # drop those who dont contain i'th priority player                                      
        max_chains = max_chains[candinates]
                                              
        i += 1
                                              
    return max_chains[0] 

def resolve_w_chain(matching, passive_matching, unassigned, w, chain):
    chain = np.array(chain) # ensure chain is an array
    # locate w
    w_loc = int(np.where(chain == w)[0].squeeze())
    # make w ending point, solution: first make w_loc
    # starting point and then roll back once 'more'
    chain = np.roll(chain, -w_loc - 1)
        
    M = len(chain)
    # skip w (M - 2)
    for j in range(M - 1):
        matching[chain[j]] = chain[j+1]
        passive_matching[chain[j]] = chain[j+1]
        # do not remove the first in the chain from unassigned (j + 1)
        if chain[j+1] in unassigned.difference({w}):
            unassigned.remove(chain[j+1])
            
    return matching, passive_matching, unassigned

### The algorithm

In [ ]:
def TTCC(A):
    N = len(A)
    # Nodes 0 - (N - 2) represent patient 1 ... (N - 1),
    # node N - 1 represents w.
    w = N - 1
    G = nx.DiGraph(directed=True)
    G.add_nodes_from(range(N))

    # patient 1 points to 56, patient 2 points to 6 etc.
    G.add_edges_from(list(enumerate(A[0])))

    matching = dict() # final matching to be returned
    # passive matching, necessary for reconstruction graph
    passive_matching = dict()
    # kidneys yet unassigned, note that this set need not be equal
    # to the set of remaining players. The first player in a w-chain
    # is not a remaining player anymore but his/her kidney need not be
    # assigned yet. (always includes w)
    unassigned = set(range(N))
    players = set(range(N-1))
    
    finished = False # boolean to tell algorithm to terminate

    while True:

        # --- CYCLE LOOP ---
        while True:
            # There exists a function to search for a single cycle
            # in networkx, however it raises an exception when no cycle found
            # and I do not whish for a try/catch block in 'production' code.
            cycles = list(nx.simple_cycles(G))
            # no more cycles found => go to w-chain step
            if len(cycles) == 0:
                break

            cycle = cycles[0]
            
            matching, unassigned = resolve_cycle(G, matching, unassigned, cycle)

            G = reconstruct_G_fully(G, matching, passive_matching, unassigned, players)

            if len(matching) == N - 1:
                finished = True
                
        if finished:
            break
        
        # At this point G is a DAG
        
        # --- w-chain part ---
        chains = find_w_chains(G, w, matching)
        max_chain = rule_e(matching, players, chains)

        matching, passive_matching, unassigned = resolve_w_chain(matching, passive_matching, unassigned, w, max_chain)
        G = reconstruct_G_fully(G, matching, passive_matching, unassigned, players)

        if len(matching) == N - 1:
            break
            
    return matching

In [ ]:
matching = TTCC(A)
# ensure that matching is valid
np.unique(list(matching.values()), return_counts=True), len(matching)

((array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  14,  15,  16,  17,  19,  20,  21,  22,  23,  24,  25,  26,
          27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
          40,  42,  43,  44,  45,  46,  47,  49,  50,  51,  52,  53,  54,
          55,  56,  57,  58,  60,  61,  62,  63,  64,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
          83,  84,  85,  86,  87,  88,  89,  90,  91,  93,  94,  95,  96,
          97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
         110, 111, 112, 114, 115, 116, 117, 118, 119]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
df = pd.DataFrame(np.array(sorted(matching.items())) + 1, columns=['patient', 'kidney']).set_index('patient').replace({120: 'w'})
df

,kidney
patient,
1,6
2,23
3,97
4,10
5,96
6,44
7,101
8,30
9,14


### Exercise (B)

### Additional auxiliary functions

In [ ]:
def create_prefix_chain(G, cycle):
    # seek player with highest priority (lowest number)
    # in the chosen cycle, start there
    cycle = np.array(cycle)
    # location of highest cycle player
    priority_loc = np.argmin(cycle)
    # make priority player starting point
    prefix_chain = list(np.roll(cycle, -priority_loc)) 
    # 'cut' the cycle at the right point
    G.remove_edge(prefix_chain[-1], prefix_chain[0])

    return G, prefix_chain

def resolve_prefix_chain(matching, passive_matching, unassigned, w, prefix_chain):
    M = len(prefix_chain)
    for j in range(M - 1):
        matching[prefix_chain[j]] = prefix_chain[j + 1]
        passive_matching[prefix_chain[j]] = prefix_chain[j + 1]
        # prefix_chain[0]'s kidney is not yet assigned
        if prefix_chain[j + 1] in unassigned.difference({w}):
            unassigned.remove(prefix_chain[j + 1])

    return matching, passive_matching, unassigned

def resolve_suffix_chain(matching, passive_matching, unassigned, w, suffix_chain):
    return resolve_prefix_chain(matching, passive_matching, unassigned, w, suffix_chain)

def reconstruct_graph(A, G, matching, unassigned, players):
# reconstruct graph carefully
    remaining_players = players.difference(matching.keys())
    if True:
        for player in remaining_players:
            if G.out_degree(player) == 1:
                successor = list(G.successors(player))[0]
                if successor not in unassigned:
                    G.remove_edge(player, successor)
                    # for preference in A[:, player]:
                    #     if preference in unassigned:
                    #         #code om te zoeken of dit tot een cycle leidt
                    #         G.add_edge(player, preference)

                    #         # check for cycles: 
                    #         break
    return G

def loose_player_best_chains(A, loose_player, unassigned, chains):
    # loop over preference, connect highest preference available
    best_chains = []
    for preference in A[:, loose_player]:
        if len(best_chains) > 0:
            break
        for chain in chains:
            if chain[0] == preference and chain[0] in unassigned:
                best_chains.append(chain)

    

    return best_chains

def roll_w_chain(w, chain):
    chain = np.array(chain) # ensure chain is an array
    # locate w
    w_loc = int(np.where(chain == w)[0].squeeze())
    # make w ending point, solution: first make w_loc
    # starting point and then roll back once 'more'
    chain = np.roll(chain, -w_loc - 1)

    return chain

def find_w_chains(G, w, matching):
    '''Finds all w-chains that contain new matchings'''
    
    # add virtual edges from w to all players such that w-chains become cycle
    G.add_edges_from([(w, x) for x in G.nodes])

    chains = []
    for chain in nx.simple_cycles(G):
        # check if chain is novel, i.e. we aren't stuck in an infinite loop
        if len(set(chain).difference(matching.keys()).difference({w})) > 0:
            chains.append(chain)
    
    # remove virtual edges
    G.remove_edges_from([(w, x) for x in G.nodes])

    # add w now since we may have empty set of nontrivial omega chains
    chains.append([w])
    return chains

In [ ]:
def modified_TTCC(A):
    N = len(A)
    # Nodes 0 - (N - 2) represent patient 1 ... (N - 1),
    # node N - 1 represents w.
    w = N - 1
    G = nx.DiGraph(directed=True)
    G.add_nodes_from(range(N))

    # patient 1 points to 56, patient 2 points to 6 etc.
    G.add_edges_from(list(enumerate(A[0])))

    matching = dict() # final matching to be returned
    # passive matching, necessary for reconstruction graph
    passive_matching = dict()
    # kidneys yet unassigned, note that this set need not be equal
    # to the set of remaining players. The first player in a w-chain
    # is not a remaining player anymore but his/her kidney need not be
    # assigned yet. (always includes w)
    unassigned = set(range(N))
    players = set(range(N-1))
    
    finished = False # boolean to tell algorithm to terminate

    while True:

        # --- CYCLE LOOP ---
        while True:
            # There exists a function to search for a single cycle
            # in networkx, however it raises an exception when no cycle found
            # and I do not whish for a try/catch block in 'production' code.
            cycles = list(nx.simple_cycles(G))
            # no more cycles found => go to w-chain step
            if len(cycles) == 0:
                break

            cycle = min(cycles, key=len)

            if len(cycle) > 3:
                # now we are in business of question (b)
                #prefix_chain = beginning of w chain

                prefix_chains = []
                # start with biggest cycle
                for cycle in reversed(sorted(cycles, key=len)):
                    G, prefix_chain = create_prefix_chain(G, cycle)
                    matching, passive_matching, unassigned = resolve_prefix_chain(matching, passive_matching, unassigned, w, prefix_chain)
                    prefix_chains.append(prefix_chain)

                # now there should not be any more cycles => DAG
                for prefix_chain in prefix_chains:
                    G = reconstruct_graph(A, G, matching, unassigned, players)

                    # find all omega chains at this moment
                    chains = find_w_chains(G, w, matching)
                    chains = [roll_w_chain(w, chain) for chain in chains]            
                    # player at the end of cut cycle
                    loose_player = prefix_chain[-1]
                    # loop over preference, connect highest preference available

                    best_chains = loose_player_best_chains(A, loose_player, unassigned, chains)
                    suffix_chain = rule_e(matching, players, best_chains)

                    # connect!
                    G.add_edge(loose_player, suffix_chain[0])
                    # pre-append loose player
                    suffix_chain = [loose_player, *suffix_chain]

                    matching, passive_matching, unassigned = resolve_suffix_chain(matching, passive_matching, unassigned, w, suffix_chain)


                
            else:
                matching, unassigned = resolve_cycle(G, matching, unassigned, cycle)

            G = reconstruct_G_fully(G, matching, passive_matching, unassigned, players)

            if len(matching) == N - 1:
                finished = True
                
        if finished:
            break
        
        # At this point G is a DAG
        
        # --- w-chain part ---
        chains = find_w_chains(G, w, matching)
        max_chain = rule_e(matching, players, chains)

        matching, passive_matching, unassigned = resolve_w_chain(matching, passive_matching, unassigned, w, max_chain)
        G = reconstruct_G_fully(G, matching, passive_matching, unassigned, players)

        if len(matching) == N - 1:
            break
            
    return matching

In [ ]:
matching = modified_TTCC(A)
# verify that matching is valid
np.unique(list(matching.values()), return_counts=True), len(matching)

((array([  0,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  19,  20,  21,  22,  23,  24,  25,  26,  28,
          29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  42,
          43,  44,  46,  47,  49,  50,  51,  52,  53,  54,  55,  56,  57,
          58,  59,  60,  61,  62,  63,  64,  65,  67,  68,  69,  70,  71,
          72,  73,  74,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
          86,  87,  88,  89,  90,  91,  93,  94,  95,  96,  97,  98,  99,
         100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
         113, 114, 115, 116, 117, 118, 119]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
df = pd.DataFrame(np.array(sorted(matching.items())) + 1, columns=['patient', 'kidney']).set_index('patient').replace({120: 'w'})
df

,kidney
patient,
1,6
2,89
3,97
4,10
5,96
6,109
7,101
8,23
9,14
